In [29]:
try:
    import pandas as pd
    import os
    import openpyxl
    import re
except:
    !pip install --upgrade -q pip
    !pip install -q pandas 
    !pip install -q openpyxl 
    !pip install -q re
    import pandas as pd
    import os
    import openpyxl
    import re

In [30]:
folder_path = os.getcwd()  # Atau ubah ke folder lain

# Ambil semua file yang berakhiran .xlsx
excel_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx")]
index = []
files = []
files_masuk = []
files_keluar = []
hasil_akhir = pd.DataFrame()

# Cetak daftar file .xlsx
for file in excel_files:
    if "Index Member" not in file:
        df = pd.read_excel(file)
        name = df.iloc[1, 6].replace(':',"").replace("-","").strip()
        evaluasi = df.iloc[2, 6].replace(':',"").strip()
        index.append(f"{name} {evaluasi}")
        
        table = []
        startrow = 0
        endrow = 0

        for i in range(len(df["Unnamed: 1"])):
            if df["Unnamed: 1"][i] == 1:
                table.append(startrow)
            try:
                if isinstance(df["Unnamed: 1"][i], int) and isinstance(df["Unnamed: 1"][i+1],str):
                    table.append(endrow+1)
            except:
                table.append(endrow+1)
            endrow+=1
            startrow+=1
        change= ["Jumlah Pra Evaluasi", "Jumlah Pasca Evaluasi", "Jumlah Keterangan", "Bobot Pra Evaluasi", "Bobot Pasca Evaluasi", "Bobot Keterangan"]
        header_masuk = [df.iloc[table[0]-2:table[0]-1, 1:4].values.tolist()[0] + change][0]
        masuk = df.iloc[table[0]:table[1], 1:12]
        masuk.columns = header_masuk
        exec(f'df_{name}_masuk = masuk')
        files.append(f'df_{name}_masuk')
        files_masuk.append(f'df_{name}_masuk')

        if len(table)>2:
            keluar = df.iloc[table[2]:table[3], 1:4]
            keluar.columns = df.iloc[table[2]-2:table[2]-1, 1:4].values.tolist()[0]
            exec(f'df_{name}_keluar = keluar')
            files.append(f'df_{name}_keluar')
            files_keluar.append(f'df_{name}_keluar')
    else:
        df_index_member = pd.read_excel(file)
        df_index_member["Index"] = df_index_member["Index"].str.replace("-", "", regex=True)

In [31]:
def Check_jumlah_naik(row):
    if row['Jumlah Keterangan'].upper() == "NAIK":
        return row['Jumlah Pasca Evaluasi'] > row['Jumlah Pra Evaluasi']
    if row['Jumlah Keterangan'].upper() == "TETAP":
        return round(row['Jumlah Pasca Evaluasi'],2) == round(row['Jumlah Pra Evaluasi'],2)
    if row['Jumlah Keterangan'].upper() == "TURUN":
        return row['Jumlah Pasca Evaluasi'] < row['Jumlah Pra Evaluasi']
    if row['Jumlah Keterangan'].upper() == "BARU":
        return (row['Jumlah Pasca Evaluasi'] > 0) & (str(row['Jumlah Pra Evaluasi']) == '-')
    return False

def Check_bobot_naik(row):
    if row['Bobot Keterangan'].upper() == "NAIK":
        return row['Bobot Pasca Evaluasi'] > row['Bobot Pra Evaluasi']
    if row['Bobot Keterangan'].upper() == "TETAP":
        return round(row['Bobot Pasca Evaluasi'],2) == round(row['Bobot Pra Evaluasi'],2)
    if row['Bobot Keterangan'].upper() == "TURUN":
        return row['Bobot Pasca Evaluasi'] < row['Bobot Pra Evaluasi']
    if row['Bobot Keterangan'].upper() == "BARU":
        return (row['Bobot Pasca Evaluasi'] > 0) & (str(row['Bobot Pra Evaluasi']) == '-')
    return False

for file in files_masuk:
    compare = file.split("_")[1]
    dict_compare = {'IDXECONOMIC30': 'ECONOMIC30'}
    if compare in dict_compare:
        compare = dict_compare[compare]
    current = df_index_member[df_index_member['Index'].str.upper() == compare]
    try:
        df_masuk = eval(file)
    except NameError:
        print(f"⚠️ DataFrame df_{file}_masuk tidak ditemukan.")
        continue 

    # Gabungkan berdasarkan kolom "Kode Efek" dan "Kode"
    exec(f'{file} = {file}.merge(current[["Kode Efek", "Weight Multiplier"]], left_on="Kode", right_on="Kode Efek", how="left")')
    exec(f'{file}["Check Kode"] = {file}["Kode Efek"] == {file}["Kode"]')
    exec(f'{file}["Check Jumlah"] = {file}["Weight Multiplier"] == {file}["Jumlah Pra Evaluasi"]')
    exec(f'{file}["Check Keterangan Jumlah"] = {file}.apply(Check_jumlah_naik, axis=1)')
    exec(f'{file}["Check Keterangan Bobot"] = {file}.apply(Check_bobot_naik, axis=1)')

    current = current.merge(df_masuk[['Kode','Jumlah Pra Evaluasi']], left_on="Kode Efek", right_on="Kode", how="left")
    current["Check Kode"] = current["Kode Efek"] == current["Kode"]
    current["Check Jumlah"] = current["Weight Multiplier"] == current["Jumlah Pra Evaluasi"]
    hasil_akhir = pd.concat([hasil_akhir, current], ignore_index=True)

    # Cek apakah DataFrame hasil kosong
    if current.empty:
        print(f"{file} not ok")


In [32]:
print(excel_files)


['1 Lamp Peng-00-BEI POP- INFOVESTA28 - Feb 2025 Mayor.xlsx', '10 Lamp Peng-00-BEI POP - IDX-MES BUMN 17 - Feb 2025 Minor.xlsx', '11 Lamp Peng-00-BEI POP- Investor33 - Feb 2025 Minor.xlsx', '12 Lamp Peng-00-BEI POP - SRIKEHATI - Feb 2025 Minor.xlsx', '13 Lamp Peng-00-BEI POP - ESGS KEHATI - Feb 2025 Minor.xlsx', '14 Lamp Peng-00-BEI POP - ESGQ KEHATI - Feb 2025 Minor.xlsx', '2 Lamp Peng-00-BEI POP- ECONOMIC30 - Feb 2025 Mayor.xlsx', '3 Lamp Peng-00-BEI POP- ABX - Feb 2025 Minor.xlsx', '4 Lamp Peng-00-BEI POP- DBX - Feb 2025 Minor.xlsx', '5 Lamp Peng-00-BEI POP- MBX - Feb 2025 Minor.xlsx', '6 Lamp Peng-00-BEI POP - ISSI - Feb 2025 Minor.xlsx', '7 Lamp Peng-00-BEI POP - JII - Feb 2025 Minor.xlsx', '8 Lamp Peng-00-BEI POP - JII70 - Feb 2025 Minor.xlsx', '9 Lamp Peng-00-BEI POP - IDX Sharia Growth - Feb 2025 Minor.xlsx', 'Index Member - Generate Report.xlsx']


In [33]:
print(index)


['IDXVESTA28 Mayor', 'IDXMESBUMN Minor', 'INVESTOR33 Minor', 'SRIKEHATI Minor', 'ESGSKEHATI Minor', 'ESGQKEHATI Minor', 'IDXECONOMIC30 Mayor', 'ABX Minor', 'DBX Minor', 'MBX Minor', 'ISSI Minor', 'JII Minor', 'JII70 Minor', 'IDXSHAGROW Minor']


In [34]:
index_list = ['IDXVESTA28 Mayor', 'IDXMESBUMN Minor', 'INVESTOR33 Minor', 
              'SRIKEHATI Minor', 'ESGSKEHATI Minor', 'ESGQKEHATI Minor', 
              'IDXECONOMIC30 Mayor', 'ABX Minor', 'DBX Minor', 'MBX Minor', 
              'ISSI Minor', 'JII Minor', 'JII70 Minor', 'IDXSHAGROW Minor']

index_list = [item.replace("Mayor", "").replace("Minor", "").strip() for item in index_list]

print(index_list)


['IDXVESTA28', 'IDXMESBUMN', 'INVESTOR33', 'SRIKEHATI', 'ESGSKEHATI', 'ESGQKEHATI', 'IDXECONOMIC30', 'ABX', 'DBX', 'MBX', 'ISSI', 'JII', 'JII70', 'IDXSHAGROW']


In [35]:
import pandas as pd

# Contoh daftar index
index_list = ['IDXVESTA28', 'IDXMESBUMN', 'INVESTOR33', 'SRIKEHATI', 
              'ESGSKEHATI', 'ESGQKEHATI', 'IDXECONOMIC30', 'ABX', 
              'DBX', 'MBX', 'ISSI', 'JII', 'JII70', 'IDXSHAGROW']

# Simpan semua DataFrame df_masuk ke dalam dictionary dengan key sebagai index_list
df_dict = {index: eval(f"df_{index}_masuk") for index in index_list if f"df_{index}_masuk" in globals()}

# Tambahkan kolom index sesuai dengan daftar index_list
for index, df in df_dict.items():
    df["Index"] = index  # Tambahkan kolom Index ke setiap DataFrame

# Gabungkan semua DataFrame
df_final = pd.concat(df_dict.values(), ignore_index=True)

df_final.loc[df_final["Jumlah Keterangan"].isin(["Naik", "Turun"]), "Jumlah Keterangan"] = "Berubah"

# Menampilkan hasil
print(df_final)


     No.  Kode Rasio Free Float Jumlah Pra Evaluasi Jumlah Pasca Evaluasi  \
0      1  AALI            0.203             7570869        4522226.744449   
1      2  ADRO           0.3202            10193680        4916420.845624   
2      3  AMRT           0.4247            21569603       23907505.388987   
3      4  ARTO           0.2693                   -       30808385.238765   
4      5  ASII           0.4499             3510130        5621693.121693   
...   ..   ...              ...                 ...                   ...   
1722  26  SILO           0.0813          1111087247            1025676024   
1723  27  SSIA           0.7269          5438190851            5472393309   
1724  28  TINS           0.3496          5285581253            5207469215   
1725  29  TKIM           0.2866           970242352            1035009855   
1726  30  TLKM           0.4714         22110742309           19891988957   

     Jumlah Keterangan Bobot Pra Evaluasi Bobot Pasca Evaluasi  \
0        

In [17]:
df_final.to_excel("hasil_gabungan.xlsx", index=False)

In [36]:
# Menampilkan hasil
print(df_final)

     No.  Kode Rasio Free Float Jumlah Pra Evaluasi Jumlah Pasca Evaluasi  \
0      1  AALI            0.203             7570869        4522226.744449   
1      2  ADRO           0.3202            10193680        4916420.845624   
2      3  AMRT           0.4247            21569603       23907505.388987   
3      4  ARTO           0.2693                   -       30808385.238765   
4      5  ASII           0.4499             3510130        5621693.121693   
...   ..   ...              ...                 ...                   ...   
1722  26  SILO           0.0813          1111087247            1025676024   
1723  27  SSIA           0.7269          5438190851            5472393309   
1724  28  TINS           0.3496          5285581253            5207469215   
1725  29  TKIM           0.2866           970242352            1035009855   
1726  30  TLKM           0.4714         22110742309           19891988957   

     Jumlah Keterangan Bobot Pra Evaluasi Bobot Pasca Evaluasi  \
0        

In [37]:
print(list(df_final.columns))

['No.', 'Kode', 'Rasio Free Float', 'Jumlah Pra Evaluasi', 'Jumlah Pasca Evaluasi', 'Jumlah Keterangan', 'Bobot Pra Evaluasi', 'Bobot Pasca Evaluasi', 'Bobot Keterangan', 'Kode Efek', 'Weight Multiplier', 'Check Kode', 'Check Jumlah', 'Check Keterangan Jumlah', 'Check Keterangan Bobot', 'Index']


In [40]:
# Ubah 'Naik' dan 'Turun' menjadi 'Berubah'
df_final.loc[df_final["Keterangan"].isin(["Naik", "Turun"]), "Keterangan"] = "Berubah"


In [45]:
import pandas as pd

# List index yang digunakan
index_list = ['IDXVESTA28', 'IDXMESBUMN', 'INVESTOR33', 'SRIKEHATI', 
              'ESGSKEHATI', 'ESGQKEHATI', 'IDXECONOMIC30', 'ABX', 
              'DBX', 'MBX', 'ISSI', 'JII', 'JII70', 'IDXSHAGROW']

# Simpan semua DataFrame df_masuk ke dalam dictionary
df_dict = {index: eval(f"df_{index}_masuk") for index in index_list if f"df_{index}_masuk" in globals()}

# Tambahkan kolom Index ke setiap DataFrame
for index, df in df_dict.items():
    df["Index"] = index  

# Gabungkan semua DataFrame
df_final = pd.concat(df_dict.values(), ignore_index=True)

# 🔹 Konversi kolom "Jumlah Pra Evaluasi" dan "Jumlah Pasca Evaluasi" ke numeric
df_final["Jumlah Pra Evaluasi"] = pd.to_numeric(df_final["Jumlah Pra Evaluasi"], errors="coerce")
df_final["Jumlah Pasca Evaluasi"] = pd.to_numeric(df_final["Jumlah Pasca Evaluasi"], errors="coerce")

# 🔹 Hitung kolom "Bobot"
df_final["Bobot"] = df_final["Jumlah Pasca Evaluasi"] - df_final["Jumlah Pra Evaluasi"]

# 🔹 Ubah NaN di kolom 'Bobot' menjadi 0
df_final["Bobot"] = df_final["Bobot"].fillna(0)

# Filter hanya kolom yang dibutuhkan
df_final = df_final[["Kode", "Jumlah Keterangan", "Index", "Bobot"]]

# Ganti nama kolom sesuai format yang diminta
df_final = df_final.rename(columns={"Jumlah Keterangan": "Keterangan"})

# 🔹 Ubah 'Naik' dan 'Turun' menjadi 'Berubah', lainnya tetap
df_final.loc[df_final["Keterangan"].isin(["Naik", "Turun"]), "Keterangan"] = "Berubah"

# 🔹 Urutkan kolom sesuai keinginan
df_final = df_final[["Kode", "Keterangan", "Bobot", "Index"]]

# Menampilkan hasil
print(df_final)


      Kode Keterangan         Bobot       Index
0     AALI    Berubah -3.048642e+06  IDXVESTA28
1     ADRO    Berubah -5.277259e+06  IDXVESTA28
2     AMRT    Berubah  2.337902e+06  IDXVESTA28
3     ARTO       Baru  0.000000e+00  IDXVESTA28
4     ASII    Berubah  2.111563e+06  IDXVESTA28
...    ...        ...           ...         ...
1722  SILO    Berubah -8.541122e+07  IDXSHAGROW
1723  SSIA    Berubah  3.420246e+07  IDXSHAGROW
1724  TINS    Berubah -7.811204e+07  IDXSHAGROW
1725  TKIM    Berubah  6.476750e+07  IDXSHAGROW
1726  TLKM    Berubah -2.218753e+09  IDXSHAGROW

[1727 rows x 4 columns]


In [46]:
df_final.to_excel("hasil.xlsx", index=False)